In [ ]:
pip install praw emoji wordcloud --upgrade

In [ ]:
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
from IPython import display
import math
import praw
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
import spacy
import sys
from wordcloud import WordCloud, STOPWORDS
import re
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")


##########################
#IMPORTANT 
#ENTER REDDIT LOGIN INFO HERE
#PLEASE SEE LINK IN READ.ME FOR HOW TO SET THIS UP
##########################


Reddit_Login = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="")

Welcome to the Reddit Music Senitiment Analyzer

Lets first start off with installing some required packages



Now that we've installed our packages and entered out reddit app information lets choose and artist or a song!



In [ ]:

#CHOOSE ARTIST HERE

Page = Reddit_Login.subreddit("Music")

Artist_or_Song = "Queen" #<----------------

In [ ]:
all_posts = []
for post in Page.search(Artist_or_Song, limit=5):
    all_posts.append(post.id)

In [ ]:
all_comments = []
for post in all_posts:
    sub = Reddit_Login.submission(id=post)
    sub.comments.replace_more(limit=0)
    for comments in sub.comments.list():
        all_comments.append(comments.body)

In this cell we will clean the comments, remove stop words and emojis and lemmatize the words



In [ ]:
string = all_comments
string = [str(i) for i in string]
string1 = ' , '.join(string)

emoji_list = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF" 
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
                      "]+", re.UNICODE)

string1_emojiless = emoji_list.sub(r'', string1)

tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|http\S+')

string_tokens = tokenizer.tokenize(string1_emojiless)

lowercase_tokens = []
for word in string_tokens:
    lowercase_tokens.append(word.lower())

nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words
words = lowercase_tokens
tokens_no_sw = [word for word in words if not word in stopwords]


lemmatizer = WordNetLemmatizer()

lem_tokens = ([lemmatizer.lemmatize(w) for w in tokens_no_sw])

string_clean = lem_tokens

In this cell we will run the VADER sentiment analysis on the comments gained from the reddit comments



In [ ]:
sentiment_analysis = SentimentIntensityAnalyzer()
sentimentized = []

for string in string_clean:
    polarity_score = sentiment_analysis.polarity_scores(string)
    polarity_score['words'] = string
    sentimentized.append(polarity_score)

pd.set_option('display.max_columns', None, 'max_colwidth', None)
df = pd.DataFrame.from_records(sentimentized)
df['label'] = 0
df.loc[df['compound'] > 0.10, 'label'] = 1
df.loc[df['compound'] < -0.10, 'label'] = -1
viz = df.loc[df['label'] != 0]
viz.head()
pos_word = list(df.loc[df['label'] == 1].words)
neg_word = list(df.loc[df['label'] == -1].words)

word_dist = list(df.loc[df['label'] != 0].words)

print(viz.label.value_counts())

Heres where we will see our results, We will produce a wordcloud of the most common words and we will show wether the comments are skewed towards negative or positive




In [ ]:
words1 = [str(w)for w in pos_word]
words2 = ' , '.join(words1)
wordcloud = WordCloud(background_color='white',stopwords = STOPWORDS).generate(words2)

words3 = [str(w)for w in neg_word]
words4 = ' , '.join(words3)
wordcloud2 = WordCloud(background_color='white',stopwords = STOPWORDS).generate(words4)

words5 = [str(w)for w in string_clean]
words6 = ' , '.join(words5)
wordcloud3 = WordCloud(background_color='white',stopwords = STOPWORDS).generate(words6)

bc = viz.label.value_counts(normalize = True)*100

chart = sns.barplot(x = bc.index, y = bc)
chart.set_xticklabels(['Neg','Pos'])
chart.set_ylabel('Percent')
plt.show



In [ ]:
plt.imshow(wordcloud3, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(wordcloud2, interpolation = 'bilinear')
plt.axis("off")
plt.show()